# cookie与session

## cookie详解

### 什么是cookie?

Cookies是一些存储在用户电脑上的小文件。它是被设计用来保存一些站点的用户数据，这样能够让服务器为这样的用户定制内容，后者页面代码能够获取到 Cookie值然后发送给服务器。比如Cookie中存储了所在地理位置，以后每次进入地图就默认定位到改地点即可。

### cookie分类

这里Cookie主要分为两种：
1. 会话Cookie：不设置过期时间，保存在浏览器的内存中，关闭浏览器，Cookie便被销毁
2. 普通Cookie: 设置了过期时间，保存在硬盘上

### cookie属性

因为最开始的cookie是网景公司定义的，后来又有了RFC版本,所以当前的Cookie有 2 个版本：
1. Version 0 
2. Version 1 

分别对应两种设置响应头的标识，分别是：Set-Cookie和Set-Cookie2,这也造成了一些属性的不同

**注意：** 常用的为Version 0

#### Version 0的属性

* NAME = Value :键值对设置要保存的Name/Value,这里的name不能和其他属性的名字一样
* Expires：过期时间
* Domain：生成该Cookie的域名
* Path： 该Cookie是在当前的哪个路径下生成
* Secure：如果设置了这个属性，那么只会在SSH连接时才会回传该Cookie

#### Version 1的属性

* Name=VALUE:键值对设置要保存的Name/Value,这里的name不能和其他属性的名字一样
* Comment：用于说明该Cookie有什么用途
* CommentURL: 该服务器为此COokie提供URI注释
* Discard: 是否在回话结束丢弃该Cookie，默认为false
* Domain: 生成该Cookie的域名
* Max-Age:最大失效时间，与Version 0不同的是这里设置的是在多少秒后失效
* Path:该Cookie是在当前的哪个路径下生成
* Port:该 Cookie 在什么端口下可以回传服务端，如果有多个端口，以逗号隔开
* Secure:如果设置了这个属性，那么只会在SSH连接时才会回传该Cookie

### 通过什么方式能获取到Cookie的内容，我能查看自己电脑上的Cookie吗？

能通过Web Server或者客户端浏览器来获取到Cookie。多数浏览器能够配置允许用户访问Cookies，但是注意不同的站点之间的Cookie是不能共享的。

打开Chrome的调试窗口可以在Resource的Cookie侧边栏查看，或者在console中输入document.cookie来查看，这属于在电脑上查看，另外一种服务器中查看可见后面的例子。

### Cookie是什么时候生成的，完整的生成、传递和使用过程是怎么样的？

写数据到Cookie中通常是在一个页面被加载的时候，比如提交按钮被按下，后台处理完请求跳转到相应页面后会把Cookie值带回来，如下是一个例子：

* 在浏览器地址栏输入了一个站点，然后浏览器会发送请求到站点的Web服务器请求该页面
* 与此同时，浏览器会在个人电脑上行查找和该站点对应的Cookie文件，如果发现了就会把里面的键值对内容全部发送给Web服务器，如果没找到则不发送。* 页面服务器接收到了Cookie的数据后，可以利用这些数据决定返回到前台的内容（更进一步是跟Session结合使用）
* 如果没键值对被Web服务器接收到，站点就知道该用户没有访问过，就会为这个访问地址新建一个ID并且发送一些键值对给前台，这些值会被放在这次响应中的Header带给浏览器，浏览器端于是有了Cookie的值
* 任何到服务的访问和数据请求都可能很对Cookie中的键值对进行修改


### 一个简单的使用Cookie例子

以python的项目flasky（一个python框架Flask的示例项目，内含完整的开发代码路径）为例，将使用git代码checkout到3b的历史节点，配置环境启动并确保`http://127.0.0.1:5000/`访问成功。
```
@app.route('/')
def index():
    try:
        print 'answer = ', request.cookies.get('answer')
        response = make_response('<h1>This document carries a cookie!</h1>')
        if request.cookies.get('answer') == None:
            response.set_cookie('answer', '42')
        else:
            response.set_cookie('answer', (request.cookies.get('answer') + '+'))

    except Exception, e:
        print e
    finally:
        return response
```        

在服务端代码中获取Cookie值，如果没有设置过就设置为“42”，否则就在原来的基础上增加“+”。每次刷新页面，可以看到后台打印的内容不停呗的累加“+”，浏览器中查看answer的值也一直在累加

### 使用Cookie存在的一些问题？

* 多个用户公用一台电脑比如网吧，那么前一个人登陆购物网站并购买物品，后一个人可以使用前一人的账号进行购物。
* 清除浏览器缓存与清除Cookie是2个操作，因此清除浏览器缓存是不会对Cookie产生影响的。清除浏览器缓存实际上是指清除临时文件，和Cookie没有关系,每个浏览器都有属于自己的＂临时文件＂，这个＂临时文件＂在本地电脑存放的路径（也就是指存放在比如Ｃ盘／Ａ文件夹／Ｂ文件夹）是不同的。
* 多台电脑之间的网站浏览记录（这里不是浏览器的历史记录）会不一致，比如这台电脑某宝告诉你你浏览过某些商品，另一台电脑可能是另外一些商品，因为不同Cookie对应的不同浏览记录。

这也是为什么需要使用账号登陆的原因，只要登陆过就能把所有记录跟后台数据库中的账号强关联起来。整理和学习居多，如有不准确，还请指正。

### 一些注意问题

在`CookieVersion0`中规定空格、方括号、圆括号、等于号、逗号、双引号、斜杠、问号、@、冒号，分号等特殊符号都不能作为Cookie的内容。

## 关于Seesion

Cookie可以让服务器端跟踪每个客户端的访问，但是每次客户端的访问都必须传回这些 Cookie，如果 Cookie 很多，这无形地增加了客户端与服务端的数据传输量，而 Session 的出现正是为了解决这个问题。

同一个客户端每次和服务端交互时，不需要每次都传回所有的Cookie值，而是只要传回一个ID这个ID是客户端第一次访问服务器的时候生成的，而且每个客户端是唯一的。这样每个客户端就有了一个唯一的ID，客户端只要传回这个ID就行了，这个ID通常是NAME为`JSESIONID`的一个Cookie。所以Session其实是利用Cookie进行信息处理的。

1. cookie和session的共同之处在于：cookie和session都是用来跟踪浏览器用户身份的会话方式。
2. cookie 和session的区别是：cookie数据保存在客户端，session数据保存在服务器端。
3. cookie不是很安全，别人可以分析存放在本地的COOKIE并进行COOKIE欺骗,如果主要考虑到安全应当使用session，当然也没有绝对的安全，只是相对cookie，session更加安全
4. session会在一定时间内保存在服务器上。当访问增多，会比较占用你服务器的性能，如果主要考虑到减轻服务器性能方面，应当使用COOKIE
5. cookie和session各有优缺点，所以将登陆信息等重要信息存放为SESSION;其他信息如果需要保留，可以放在COOKIE中

## cookie与session区别

1. 由于HTTP协议是无状态的协议，所以服务端需要记录用户的状态时，就需要用某种机制来识具体的用户，这个机制就是Session.典型的场景比如购物车，当你点击下单按钮时，由于HTTP协议无状态，所以并不知道是哪个用户操作的，所以服务端要为特定的用户创建了特定的Session，用用于标识这个用户，并且跟踪用户，这样才知道购物车里面有几本书。这个Session是保存在服务端的，有一个唯一标识。在服务端保存Session的方法很多，内存、数据库、文件都有。集群的时候也要考虑Session的转移，在大型的网站，一般会有专门的Session服务器集群，用来保存用户会话，这个时候 Session 信息都是放在内存的，使用一些缓存服务比如Memcached之类的来放 Session。

2. 思考一下服务端如何识别特定的客户？这个时候Cookie就登场了。每次HTTP请求的时候，客户端都会发送相应的Cookie信息到服务端。实际上大多数的应用都是用 Cookie 来实现Session跟踪的，第一次创建Session的时候，服务端会在HTTP协议中告诉客户端，需要在 Cookie 里面记录一个Session ID，以后每次请求把这个会话ID发送到服务器，我就知道你是谁了。有人问，如果客户端的浏览器禁用了 Cookie 怎么办？一般这种情况下，会使用一种叫做URL重写的技术来进行会话跟踪，即每次HTTP交互，URL后面都会被附加上一个诸如 sid=xxxxx 这样的参数，服务端据此来识别用户。

3. Cookie其实还可以用在一些方便用户的场景下，设想你某次登陆过一个网站，下次登录的时候不想再次输入账号了，怎么办？这个信息可以写到Cookie里面，访问网站的时候，网站页面的脚本可以读取这个信息，就自动帮你把用户名给填了，能够方便一下用户。这也是Cookie名称的由来，给用户的一点甜头。

## 总结一下：

* Cookie是客户端（浏览器）保存用户信息的一种机制，用来记录用户的一些信息，也是实现Session的一种方式。
* Session是在服务端保存的一个数据结构，用来跟踪用户的状态，这个数据可以保存在集群、数据库、文件或内存中；
* session 的运行依赖 session id，而 session id 是存在 cookie 中的，也就是说，如果浏览器禁用了 cookie ，同时 session 也会失效（但是可以通过其它方式实现，比如在 url 中传递 session_id）
* session 可以放在 文件、数据库、或内存中都可以。
* 用户验证这种场合一般会用 session, 因此，维持一个会话的核心就是客户端的唯一标识，即 session id

### cookie sessionStorage localStorage有什么不同

* cookie存储在客户端，可以发送给服务器，数据大小限制为4K

* sessionStorage,localStorage存储在本地，不可以发送给服务器，数据大小为5M

* localStorage只能手动清除数据

* sessionStorage关闭会话窗，数据就被清除了